<a href="https://colab.research.google.com/github/Saputoa21/Machine-Translation/blob/main/filterMT_AdvancedMT_2025W.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://github.com/ymoslem/Adaptive-MT-LLM-Fine-tuning/blob/main/Data-Processing-Adaptive-MT.ipynb
#https://github.com/ymoslem/MT-Preparation/tree/main

In [17]:
!git clone https://github.com/ymoslem/MT-Preparation.git

fatal: destination path 'MT-Preparation' already exists and is not an empty directory.


In [18]:
%cd MT-Preparation

/content/MT-Preparation


In [19]:
!pip3 install --user -r requirements.txt

In [20]:
%cd ..

/content


In [21]:
!ls

drive		   UN.en-ru.train.en-filtered.en
model_cache	   UN.en-ru.train.en-filtered.en.semantic.en
MT-Preparation	   UN.en-ru.train.ru
sample_data	   UN.en-ru.train.ru-filtered.ru
UN.en-ru.train.en  UN.en-ru.train.ru-filtered.ru.semantic.ru


In [7]:
!python3 MT-Preparation/filtering/filter.py train.en-de.en train.en-de.de en de

Dataframe shape (rows, columns): (50000, 2)
--- Rows with Empty Cells Deleted	--> Rows: 49997
--- Duplicates Deleted			--> Rows: 49997
--- Source-Copied Rows Deleted		--> Rows: 49997
--- Too Long Source/Target Deleted	--> Rows: 48399
--- Too Short Source/Target Deleted	--> Rows: 48321
--- HTML Removed			--> Rows: 48321
--- Rows will remain true-cased		--> Rows: 48321
--- Rows with Empty Cells Deleted	--> Rows: 48315
--- Rows Shuffled			--> Rows: 48315
--- Source Saved: train.en-de.en-filtered.en
--- Target Saved: train.en-de.de-filtered.de


In [8]:
!python3 MT-Preparation/filtering/semantic_filter.py train.en-de.en-filtered.en train.en-de.de-filtered.de en de 10000 0.45

2025-10-28 14:04:38.833889: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761660278.853572    1551 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761660278.859542    1551 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761660278.874791    1551 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761660278.874840    1551 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761660278.874844    1551 computation_placer.cc:177] computation placer alr

In [9]:
!wc -l *

wc: model_cache: Is a directory
       0 model_cache
wc: MT-Preparation: Is a directory
       0 MT-Preparation
wc: sample_data: Is a directory
       0 sample_data
   50000 train.en-de.de
   48315 train.en-de.de-filtered.de
   47614 train.en-de.de-filtered.de.semantic.de
   50000 train.en-de.en
   48315 train.en-de.en-filtered.en
   47614 train.en-de.en-filtered.en.semantic.en
  291858 total


# **My corpus text**

I have chosen MultiUN (v1) corpus with the language pair EN-RU and size of 11,654,416 sentences. I took the first 100k sentences to create the corpus, which will be splitted in the following notebooks into train, val and test sets.

As tzhe corpus (txt.files) was too big to load into Colab, I uploaded the gz.files to my Google Drive to open it here and take only some sentences.

The created files are:

*   UN-train.en-ru.en
*   UN-train.en-ru.ru
*   UN.en-ru.train.en-filtered.en
*   UN.en-ru.train.ru-filtered.ru
*   List item
*   List item


Link: https://opus.nlpl.eu/MultiUN/en&ru/v1/MultiUN


## Corpus creation

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
file_ru = "/content/drive/MyDrive/Colab Notebooks/ru.txt.gz"
file_en = "/content/drive/MyDrive/Colab Notebooks/en.txt.gz"

In [9]:
import gzip

def extract_first_n_lines(gz_path, n_lines=100000):
    lines = []
    with gzip.open(gz_path, 'rt', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= n_lines:
                break
            lines.append(line)
    print(f"Extracted {len(lines):,} lines from {gz_path}")
    return lines

In [10]:
train_ru = extract_first_n_lines(file_ru, n_lines=100000)
train_en = extract_first_n_lines(file_en, n_lines=100000)

Extracted 100,000 lines from /content/drive/MyDrive/Colab Notebooks/ru.txt.gz
Extracted 100,000 lines from /content/drive/MyDrive/Colab Notebooks/en.txt.gz


In [11]:
print(len(train_ru))
print(len(train_en))

100000
100000


In [12]:
def save_bilingual_splits(prefix="UN", src_lang="en", tgt_lang="ru",
                          train_src=None, train_tgt=None):

    def save(name, src_data, tgt_data):
        src_file = f"{prefix}.{src_lang}-{tgt_lang}.{name}.{src_lang}"
        tgt_file = f"{prefix}.{src_lang}-{tgt_lang}.{name}.{tgt_lang}"

        with open(src_file, "w", encoding="utf-8") as f_src:
            f_src.writelines(src_data)
        with open(tgt_file, "w", encoding="utf-8") as f_tgt:
            f_tgt.writelines(tgt_data)

        print(f"Saved {src_file} ({len(src_data):,} lines)")
        print(f"Saved {tgt_file} ({len(tgt_data):,} lines)")

    if train_src and train_tgt:
        save("train", train_src, train_tgt)

In [13]:
save_bilingual_splits(
    prefix="UN",
    src_lang="en",
    tgt_lang="ru",
    train_src=train_en, train_tgt=train_ru,
)

Saved UN.en-ru.train.en (100,000 lines)
Saved UN.en-ru.train.ru (100,000 lines)


## Corpus preparation

In [14]:
!python3 MT-Preparation/filtering/filter.py UN.en-ru.train.en UN.en-ru.train.ru en ru

Dataframe shape (rows, columns): (100000, 2)
--- Rows with Empty Cells Deleted	--> Rows: 100000
--- Duplicates Deleted			--> Rows: 99969
--- Source-Copied Rows Deleted		--> Rows: 99968
--- Too Long Source/Target Deleted	--> Rows: 48695
--- Too Short Source/Target Deleted	--> Rows: 48664
--- HTML Removed			--> Rows: 48664
--- Rows will remain true-cased		--> Rows: 48664
--- Rows with Empty Cells Deleted	--> Rows: 48664
--- Rows Shuffled			--> Rows: 48664
--- Source Saved: UN.en-ru.train.en-filtered.en
--- Target Saved: UN.en-ru.train.ru-filtered.ru


After anaylsing the semantically filtered corpus, I got surprisingly to few sentence, ie.e fewer than 100. Therefore I tried several attemts with different batches and treshholds and stopped at this variant, as it gave already "enough" sentences for train corpus

In [15]:
!python3 MT-Preparation/filtering/semantic_filter.py UN.en-ru.train.en-filtered.en UN.en-ru.train.ru-filtered.ru  en ru 1000 0.30

2025-10-28 15:07:45.739793: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761664065.759413    4587 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761664065.765468    4587 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761664065.781391    4587 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761664065.781422    4587 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761664065.781427    4587 computation_placer.cc:177] computation placer alr